In [18]:
import os
os.environ["OPEN_API_KEY"] = "your_api_key"

In [19]:
OPENAI_API_KEY="your_api_key"

In [20]:
import warnings
warnings.filterwarnings("ignore")

# Building a Vector Database 

Step 1 is preparing a database that can be easily queried - this involves 3 steps 

1. Collect and load data
2. Chunk documents
3. Embed and store chunks

In [21]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [22]:
loader = CSVLoader(file_path="/Users/sarah_prakriti_peters/Documents/Instalily/data/fridge_data.csv",
                   csv_args={"delimiter": ","})
data = loader.load()

In [23]:
data

[Document(page_content=': 0\nPart_Number: PS11701542\nManufacturer_Part_Number: EDR1RXD1\nManufacturer: Whirlpool\nTitle: Fridge Refrigerator Ice and Water Filter\nDescription: This is an OEM replacement ice and water filter for your refrigerator. In side-by-side refrigerators, it is found in the top right corner or the bottom left grille. In bottom-freezer refrigerators, the filter is in the bottom left grille. In top-freezer units, the filter is located in the middle of the refrigerator. This NSF certified filter reduces 28 contaminants including pharmaceuticals, metals, minerals, pesticides, cysts, industrial chemicals, Class I Particulates and chlorine. It is made up of coconut carbon block and has three layers of filtration. The first layer is particle filtration that helps block larger contaminants. The second layer is micro-filtration, which prevents medium-sized impurities from passing. The third layer is called absorption filtration, which absorbs microscopic-sized pollutants.

In [24]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(data)

In [25]:
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(openai_api_key="your_api_key"),
    by_text = False
)

embedded weaviate is already listening on port 8079


{"level":"info","msg":"Created shard langchain_2a6536a4d5f0472190333a1d9d81e8d0_LxmWeNBv7VbH in 1.259542ms","time":"2024-03-01T15:20:07-05:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-01T15:20:07-05:00","took":40875}


In [26]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Weaviate', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.weaviate.Weaviate object at 0x154c01820>)

# Setting up a chat prompt 

I want the agent to be able to answer all basic questions about refrigerators

In [27]:
template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [28]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key="your_api_key")

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "The ice maker on my Whirlpool fridge is not working. How can I fix it?"
rag_chain.invoke(query)

'To fix the ice maker on your Whirlpool fridge, you may need to replace the water filter. The water filter purifies water going to the ice maker and can fix issues like reduced water flow or no ice production. Replacing the water filter every 6 months is recommended for optimal performance.'

In [29]:
query = "How can I install part number PS11752778?"
rag_chain.invoke(query) 

'To install part number PS11752778, grab the old bin with both hands and lift it away from the door, then line up the new bin where the old one was and drop it into place. No tools are needed for installation. Good luck with your repair!'

In [30]:
query = "How expensive is part number PS11752778? Also who is the manufacturer?"
rag_chain.invoke(query)

'Part number PS11752778 is priced at $47.83 and is manufactured by Whirlpool.'

In [31]:
query = "How expensive is part number PS11752778? Also who is the manufacturer? And is it currently in stock? Also any idea what parts it can replace?"
rag_chain.invoke(query)

'The part number PS11752778 is priced at $47.83, manufactured by Whirlpool, and currently in stock. It can replace parts such as AP6019471, 2171046, 2171047, and more.'

In [32]:
query = "How do most customers rate 242126602?"
rag_chain.invoke(query)

'Most customers rate 242126602 with an average product rating of 4.8 and an average repair rating of 4.4 out of 5.0 based on 20 reviews.'

In [33]:
query = "My refrigerator makes noise. Which part should be replaced? Also, how expensive is that part?"
rag_chain.invoke(query)

'The part that should be replaced in your refrigerator is the water filter. The price for the water filter is $90.73. It is recommended to replace the water filter every 6 months for optimal performance.'

In [34]:
query = "Is the refrigerator crisper drawer with humidity control necessary? What kinds of problems will it fix?"
rag_chain.invoke(query)

'The refrigerator crisper drawer with humidity control is necessary to regulate humidity levels and keep vegetables and fruits fresh. It can fix problems like difficulty sliding the drawer in and out due to damage or breakage. Replacing the drawer can help maintain the freshness of produce stored in the refrigerator.'

{"level":"info","msg":"Created shard langchain_d3d97aed63af43a181322181964a5b79_xs6ZT2RBJYsE in 1.023917ms","time":"2024-03-01T15:33:42-05:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-01T15:33:42-05:00","took":40875}
{"level":"info","msg":"Created shard langchain_f11607d01054423682f965a1e67e6a9a_Qj6xnciTnkLD in 1.105833ms","time":"2024-03-01T15:33:51-05:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-01T15:33:51-05:00","took":40750}
{"level":"info","msg":"Created shard langchain_f5685cab3580406cbe0e31d55beb0190_t8UJp3XNw2VL in 2.911583ms","time":"2024-03-02T12:30:33-05:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-02T12:30:33-05:00","took":193500}
{"level":